In [18]:
def jobs_manager():
    from IPython.lib.backgroundjobs import BackgroundJobManager
    from IPython.core.magic import register_line_magic
    from IPython import get_ipython

    jobs = BackgroundJobManager()

    @register_line_magic
    def job(line):
        ip = get_ipython()
        jobs.new(line, ip.user_global_ns)

    return jobs
import sys
import argparse
import random
from nile.api.v1 import (
    clusters,
    Record,
    filters as nf,
    aggregators as na,
    grouping as ng,
    extractors as ne,
    statface as ns
)
from qb2.api.v1 import (
    filters as sf,
    extractors as se,
    resources as sr
)
import nile
from urlparse import urlparse
import json
from itertools import product
from nile.api.v1 import datetime as nd
import re
import pandas as pd
import random
import string
from datetime import datetime

In [37]:
cluster = clusters.YT(proxy="hahn.yt.yandex.net", pool="voice") \
                .env(templates=dict(
                                job_root=('home/voice/talamable/VA-274'),
                                dates='{2018-08-20..2018-08-26}'))

In [40]:
def parse_us(groups):
    import libra
    for key, records in groups:
        uuid = key.key
        if not uuid.startswith('uu/'):
            continue
        try:
            session = libra.ParseSession(records, 'blockstat.dict')
        except:
            continue
        for r in session:
            urls = {}
            if not r.IsA('TMobileAppYandexWebRequest'):
                continue
            if not ('query_source=dialog' in r.FullRequest):
                continue
            commercial = float(r.RelevValues.get('cm2', -1))
            if commercial < 0.2:
                continue
            results = r.GetMainBlocks()
            for block in results:
                res = block.GetMainResult()
                if res.IsA('TWebResult'):
                    url = urlparse(res.Url).netloc
                    if url.startswith('www.'):
                        url = '.'.join(url.split('.')[1:])
                    urls[url] = urls.get(url,0) + len(block.GetClicks())
            yield Record(query=r.Query,
                         cm=commercial,
                         urls=json.dumps(urls)
                        )

In [41]:
job = cluster.job()
us = job.table('//user_sessions/pub/search/daily/@dates/clean')
reqs = us.groupby('key').sort('subkey').reduce(parse_us,
                                                   files=[nile.files.RemoteFile('statbox/statbox-dict-last/blockstat.dict'),
                                                          nile.files.RemoteFile('statbox/resources/libra.so')
                                                         ],
                                                   memory_limit=4000
                                                  ) \

            .put('$job_root/commercial_week')
job.run()

VBox()

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
def unfold_hosts(records):
    for rec in records:
        for host, clicks in json.loads(rec['urls']).iteritems():
            yield Record(query=rec['query'],
                         cm=rec['cm'],
                         host=host, clicks=clicks
                        )
job = cluster.job()
hosts = job.table('$job_root/commercial_week') \
            .filter(nf.custom(lambda x: x>0.2, 'cm')) \
            .map(unfold_hosts) \
            .groupby('query','host') \
            .aggregate(cm=na.median('cm'),
                       clicks=na.sum('clicks')
                      ) \
            .put('$job_root/commercial_week_hosts')
job.run()

VBox()